<a href="https://colab.research.google.com/github/ednavivianasegura/H_HSGP4_Ariadna/blob/main/Ariadna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# @title **Clonar carpeta GitHub** (Para acceder a módulos y datasets) { form-width: "5%", display-mode: "form" }
# Clona el repositorio
try:
    !git clone https://github.com/ednavivianasegura/H_HSGP4_Ariadna.git
except:
    pass

# Cambiar al directorio "Curso_PLN"
os.chdir("H_HSGP4_Ariadna")

In [ ]:
!git clone https://gitlab.com/libeigen/eigen.git
# !sudo mv eigen /usr/local/include/


In [ ]:
!pip install sinfo

In [ ]:
import os, time, sys, gc, random, json, re, shutil
from distutils.util import strtobool
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import subprocess
import platform
#Validar requisitos del script
from sinfo import sinfo
import FuncionesAriadnaDL_Noaleatorio as FAVCI

Plataforma=platform.platform()
print(Plataforma)
print(sinfo())

In [ ]:
# @title Entradas:
# Modeling=True if Tensorflow is to be used (The models are created), Modeling=False otherwise
Modeling = True # @param {type:"boolean"}

fore_set = ["train","test"] # @param ["[\"train\"]", "[\"valid\"]", "[\"test\"]", "[\"train-valid\"]", "[\"valid-test\"]", "[\"train-valid-test\"]", "[\"train\",\"test\"]", "[\"train\",\"valid\"]", " [\"train\",\"valid\",\"test\"]", "[\"train\",\"test\",\"train-valid-test\"]", "[\"train\",\"test\",\"valid-test\",\"train-valid-test\"]", "[\"train-test\"]", "[\"train\",\"train-test\"]", "[\"train\",\"valid-test\",\"test\"]"] {type:"raw"}

# In BasesAModelar you can set up an arrangement of bases to perform the learning process.
# Intervalo = True is to define an interval, Intervalo = False is to indicate that it is an array of specific series.

Intervalo = False # @param {type:"boolean"}
BasesAModelar = [1,2]

# Type of variables:
var_set = "hill" # @param ["hill", "dln", "orb", "equi", "cart"]

# Especific name for the experiment (to be included in folder name)
experimento = "Prueba" # @param {type:"string"}

fachl = 0.5 # @param {type:"slider", min:0, max:10, step:0.1}

#TYPE OF HYPER-PARAMETER SEARCH
method = "Cartesian" # @param ["Cartesian", "Random", "Especific model"]

# Effective (without starting vector)
train_set = 0 # @param {type:"number"}

valid_set = 0 # @param {type:"number"}

test_set = 0 # @param {type:"number"}

# Resolution: [min]
resol = 10  # @param {type:"number"}

# Cross-val (if nfolds!:0 -> valid_set:0)
nfolds = 0   # @param {type:"number"}

plot_width = 10 # @param {type:"number"}
plot_height = 8 # @param {type:"number"}

# Remove trend
rem_trend = False # @param {type:"boolean"}

path_data = "data/" # @param {type:"string"}


var_model = "theta" # @param {type:"string"}

var_order= ["t","r","theta","v","R","THETA","N"]

var_lab_unit = "rad" # @param {type:"string"

#############################################################################################################
###### SAMPLING CRITERIA
#############################################################################################################
#Sampling proportional to the stratum size "SampleMethod":1 is chosen, "SampleMethod":0 otherwise.
SampleMethod = True  # @param {type:"boolean"}
# if SampleMethod=1, one of the hyperparameters should be chosen for stratification, the variable name, must be equal to the name of the hyperparameter defined in ParametersDL
VariableToStratify = "opt" # @param {type:"string"}

##########################################################################################################
###### HYPER-PARAMETERS DEEP LEARNING
##########################################################################################################
Parametros = {
#ACTIVATION FUNCTION
  "FuncAct":["tanh","linear","elu","relu"],
#BATCH SIZE
  "BachSize":[64,128,256,512],
#NUMBER OF NEURONS IN THE INPUT LAYER
  "NumNeurons":[8,16,32,64,128,256],
#OPTIMIZERS
  "opt":["Adam", "Rmsprop","Adadelta", "Nadam"],
#loss:
  "loss":["mape"],
#Learning rate:
  "LR":[1e-05],
#Monitor
  "monitor" : ["val_loss"],
#NUMBER OF HIDDEN LAYERS
  "HiddenLayers":[2]
    }

#Specific configuration:
CONFIG =[
{"BachSize": 64, "NumNeurons": 16, "opt": "Rmsprop", "loss": "mape", "LR": 0.0001, "monitor": "val_loss", "FuncAct0": "tanh", "FuncAct1": "elu", "HiddenLayers": 2},
{"BachSize": 256, "NumNeurons": 64, "opt": "Nadam", "loss": "mape", "LR": 0.0001, "monitor": "val_loss", "FuncAct0": "linear", "FuncAct1": "tanh", "HiddenLayers": 2},
{"BachSize": 256, "NumNeurons": 32, "opt": "Nadam", "loss": "mape", "LR": 0.0001, "monitor": "val_loss", "FuncAct0": "linear", "FuncAct1": "tanh", "HiddenLayers": 2}
]


# Keplerian period: [h] (0 for AUTO)
kep_period      = 0

# Possibilities: "num", "span" [h], "rev" [rev]
input_unit = "rev"

inputI = 2

output_num=1

sets_unit=input_unit

# Step betconsecutive vectors (1 to maximize no. vectors)
vect_step      = 1

#max number of epochs
epocas = 500  # @param {type:"number"}
#Patience to find number of Epochs
stop_tolerance = 60  # @param {type:"number"}
#Number of models you create with the same architecture to pick the minimum,  must be greater than 0
n_repeats = 10   # @param {type:"number"}
verbose = 0 # @param ["0", "1", "2"] {type:"raw"}


Functions            ={
    "relu":"{if(x<0) return 0.0;  else return 1.0*(x);});",
    "tanh":"{return tanh(x);});",
    "linear":"{return 1.0*(x);});",
    "elu":"{if(x<0) return 1.0*(exp(x)-1); else return 1.0*(x);});",
    "sigmoid":"{return 1/(1+exp(-x));});"
    }
t_dist = [2,4,6,8,10,12,14,20,30]
var_angular= True # @param {type:"boolean"}
PathEIGEN = "eigen" # @param {type:"string"}
Forecast= True # @param {type:"boolean"}


In [ ]:
I_CreaInfo=time.time()
if Intervalo==True:
    BasesAModelar=range(BasesAModelar[0],BasesAModelar[1]+1)
else:
    BasesAModelar=BasesAModelar

for h1 in BasesAModelar:

    print(var_set+"-"+str(h1)+" Time series modelling.")
    ##########################################################################################################
    ###### NAME OF EPHEMERID TO BE STUDIED
    ##########################################################################################################
    id_expM = var_set+str(h1)
    ##########################################################################################################
    ###### CREATE NAME OF DIRECTORY WITH PARAMETERS
    ##########################################################################################################
    experiment=str(experimento)+"_"+"Fachl"+str(fachl) +"Me"+str(method)+"TrS"+str(train_set) + "VaS"+str(valid_set)+"TeS"+str(test_set)+"Res"+str(resol)+"Nf"+str(nfolds)+"Rt"+str(rem_trend) +"VarS"+str(var_set)+"Ep"+str(epocas)+"St"+str(stop_tolerance)+"VarM"+str(var_model)
    ###########################################################################################################
    ###### Create folder name
    ###########################################################################################################
    Conid_exp = id_expM + experiment
    cwd=os.getcwd()
    cwd2=cwd
    ##########################################################################################################
    ###### Configuration of files and folders
    ##########################################################################################################
    try:
      os.stat('KerasPruebasFinal/')
    except:
      os.mkdir('KerasPruebasFinal/')

    path= 'KerasPruebasFinal/'+Conid_exp+'/'

    path_results           = "results/"
    path_out               = "output/"
    ResumenEjecucion       = path+"output/Log.txt"
    path_plot              = "figs/"
    path_models            = "Modelos/"
    path_cpp               = "cpp/"
    file_obs               = 'obs'+id_expM+'.out'
    file_appr              = 'approx'+id_expM+'.out'
    file_BasicPlot         = id_expM +"_BasicPlot"
    file_plot_Decomp       = id_expM +"_Decomp"
    file_plot_Complete     = id_expM +"_Completo"
    fileCandidato          = path+path_results+'Candidatos.csv'
    path_path_models       = path+path_models
    path_file_obs          = path_data+"OBS/"+file_obs
    path_file_appr         = path_data+"APPROX/"+file_appr
    path_file_BasicPlot    = path+path_plot + file_BasicPlot +'.pdf'
    fileplotDetrending     = path+path_plot + file_plot_Decomp+".pdf"
    fileplotComplete       = path+path_plot + file_plot_Complete+".pdf"
    directorio             = ['figs','Modelos','output','results', "cpp"]

    #Grid Comfigurations:
    hp_all=FAVCI.DeepLerningModelConfigs(Parametros)
    NModTotal=len(hp_all)
    random.seed(2022)
    hp_all=random.sample(hp_all, len(hp_all))

    if method=="Cartesian":
        NModels=NModTotal
        hp_all=hp_all
        Metodo="Cartesiana"
    elif method=="Random":
        Metodo="Aleatoria"
        print("Hiper-parameter to sampling: "+ VariableToStratify)
        ListaDeOpciones=Parametros[VariableToStratify]
        hp_all=[y for x in ListaDeOpciones for y in hp_all if x==y[VariableToStratify]]
        StrVar=[str(hp_all).count("'"+VariableToStratify+"': '"+str(n)+"'") if type(n)==str else str(hp_all).count("'"+VariableToStratify+"': "+str(n)) for n in ListaDeOpciones]

        if SampleMethod==0:
            if NModels< len(StrVar):
                NModels=len(StrVar)
            else:
                NModels=FAVCI.roundBy(NModels, len(StrVar))
            indices=[]
            nPerHipePar=round(NModels/len(StrVar))
            a=0
            for i in StrVar:
                random.seed(2022)
                indices=indices+random.sample(range(a, i+a), nPerHipePar)
                a=i+a
            hp_all=[hp_all[index] for index in indices]
        else:
            indices=[]
            nPerHipePar=[round(NModels*(x/sum(StrVar))) for x in StrVar]
            a=0
            zipped = list(zip(StrVar,nPerHipePar))
            for i in zipped:
                print(i)
                random.seed(2022)
                X=random.sample(range(a, i[0]+a), i[1])
                indices=indices+X
                a=i[0]+a
            # indices.sort()
            hp_all=[hp_all[index] for index in indices]


    elif method==3:
        Metodo="Modelo Específico"
        # hp_all = [CONFIG]
        hp_all = CONFIG
        NModels=len(hp_all)

    print("Total number of possible models: "+ str(NModTotal)+"\n"+
          "Number of models to find: "+ str(len(hp_all)))

    ###########################################################################################################
    ###### FOLDER CREATION AND VALIDATION
    ###########################################################################################################
    # Check if a folder with the same name as the current experiment already exists.
    # If not, a folder with that name is created.
    ###########################################################################################################

    Verif=path_results

    if os.path.exists(path):

    ###############################################################
    # It is verified that the existing folder is of a complete or
    # finished experiment, otherwise it is deleted.
    # experiment, otherwise the folder is deleted.
    ###############################################################

        if os.path.exists(path+Verif) and len(path_path_models)==NModTotal:

            sub=1
            repeticion=""
            NuevoArchivo="False"


            while(NuevoArchivo=="False"):

    #############################################################################
    # If the existing folder has a completed experiment
    # then folders are searched for folders with the same experiment but
    # performed later. What is identified by (i), if found, is
    # verify that they are complete. Otherwise, they are deleted and the current
    # is assigned that name. "os.path.exists(path)" is checked
    # again since in each iteration of while, it changes due to "sub".
    #############################################################################

                if  os.path.exists(path):

                    if len(os.listdir(path+Verif))>0:
                        repeticion="(" +str(sub)+")" +'/'
                        path= 'KerasPruebasFinal/'+Conid_exp+repeticion
                        sub+=1

                    else:

                        print(f"Borrando directorio vacio 1 {path}."+'\n')
                        shutil.rmtree(path)
                        os.mkdir(path)
                        os.chdir(path)


                        for dir in directorio:
                            try:
                                os.stat(dir)
                            except:
                                os.mkdir(dir)

                        NuevoArchivo="True"
                        os.chdir(cwd2)

                else:

                    os.mkdir(path)
                    os.chdir(path)


                    for dir in directorio:
                            try:
                                os.stat(dir)
                            except:
                                os.mkdir(dir)
                    NuevoArchivo="True"
                    os.chdir(cwd2)



        elif os.path.exists(path+Verif) and len(path_path_models)<NModTotal:
            pass
        else:

            print(f"Borrando directorio vacio 2 {path}."+'\n')
            shutil.rmtree(path)
            os.mkdir(path)
            os.chdir(path)


            for dir in directorio:

                try:
                    os.stat(dir)
                except:
                    os.mkdir(dir)

            os.chdir(cwd2)
    else:

        os.mkdir(path)
        os.chdir('KerasPruebasFinal/'+Conid_exp)

        for dir in directorio:

            try:
                os.stat(dir)
            except:
                os.mkdir(dir)

        os.chdir(cwd2)


    if not os.path.isfile(path+path_out+"hp_all.csv"):
        pd.DataFrame(hp_all).to_csv(path+path_out+"hp_all.csv")
    #########################################################################
    #                End of Folder Verification                             #
    #########################################################################
    if not os.path.isfile(ResumenEjecucion):
        Start=str(
          "#############################################################################\n"+
          "Start modeling: \n"+
          "#############################################################################\n"+
          "Start time: "+ str(datetime.now()) +"\n"+
          "Folder Name: "+ str(Conid_exp) +"\n"+
          "Grid Search Method: " + str(Metodo) + "\n"+
          "Number of models: "+ str(NModels) +"\n"+
          "Maximum number of epochs: "+str(epocas)+ "\n"+
          "Stop tolerance: "+ str(stop_tolerance) +"\n"+
          "#############################################################################\n"+
          "#############################################################################\n"
          )
        with open(ResumenEjecucion, 'a') as f:
            print(Start
          , file=f)
        print(Start)

    ##########################################################################################################
    ###### CREATING DATABASES
    ##########################################################################################################
    I_bases=time.time()
    Bases=FAVCI.creaBases(var_order,path_file_obs,path_file_appr,resol,kep_period,var_set,input_unit,inputI,
                          output_num,nfolds,sets_unit,train_set,test_set,var_model,rem_trend,vect_step,
                          ResumenEjecucion,valid_set)
    ind_train=Bases[0]
    train_seq=Bases[1]
    train_seq_num=Bases[2]
    train_trend_seq=Bases[3]
    train_vect=Bases[4]
    ind_valid=Bases[5]
    ind_valid0=Bases[6]
    valid_seq=Bases[7]
    valid_seq_num=Bases[8]
    valid_trend_seq=Bases[9]
    valid_vect=Bases[10]
    ind_test=Bases[11]
    ind_test0=Bases[12]
    test_seq=Bases[13]
    test_trend_seq=Bases[15]
    appr=Bases[16]
    obs=Bases[17]
    error=Bases[18]
    error_comps=Bases[19]
    error_trend=Bases[20]
    errorBU=Bases[21]
    input_num=Bases[22]

    print('Time creating data '+str(int(time.time()-I_bases))+' sec'+'\n')

    ##########################################################################################################
    ###### MAIN GRAPHICS
    ##########################################################################################################
    g_time=time.time()

    if not os.path.isfile(path_file_BasicPlot):
        FAVCI.BasicPlot(var_model,path_file_BasicPlot,plot_width,plot_height,train_seq,input_num,
                        output_num,train_seq_num,test_seq,nfolds,valid_seq,var_lab_unit)

    if (rem_trend):

        train_seq_t  =  pd.concat([train_seq['t'], train_seq[var_model]-train_trend_seq[var_model]], axis=1)
        valid_seq_t  =  pd.concat([valid_seq['t'], valid_seq[var_model]-valid_trend_seq[var_model]], axis=1)
        test_seq_t   =  pd.concat([test_seq['t'], test_seq[var_model]-test_trend_seq[var_model]], axis=1)

        if not os.path.isfile(fileplotDetrending):
            FAVCI.plot_decomp(fileplotDetrending,error_comps)
        if not os.path.isfile(fileplotComplete):
            FAVCI.plot_complete(fileplotComplete,errorBU,var_model,error_trend,error,var_lab_unit)

        FAVCI.BasicPlot(var_model,path_file_BasicPlot,plot_width,plot_height,train_seq_t,input_num,
                        output_num,train_seq_num,test_seq_t,nfolds,valid_seq_t,var_lab_unit)

    print('Time main graphics '+str(int(time.time()-g_time))+' sec'+'\n')

    ##########################################################################################################
    ###### Call KERAS.SERIE function (Tensorflow + keras)
    ##########################################################################################################

    collected = gc.collect()
    print("Garbage collector start: collected %d objects." % (collected))


    ###########################################################################################################
    ###########################################################################################################
    #### Modelling ####
    I_Modelado=time.time()
    if Modeling==1:
        for lista in hp_all:

            Candidatos= FAVCI.KerasSerie(train_vect,valid_vect,fileCandidato,path_path_models,
                           n_repeats,lista,input_num,fachl,stop_tolerance,epocas,
                          nfolds,path,path_plot,verbose,var_order,path_file_obs,
                          path_file_appr,resol,kep_period,var_set,input_unit,inputI,
                          output_num,sets_unit,train_set,valid_set,test_set,var_model,
                          var_angular,rem_trend,vect_step,ResumenEjecucion,t_dist,
                          PathEIGEN,Functions,path_cpp)
            print("Length Candidates: "+str(Candidatos.shape[0]))
            collected = gc.collect()
            print("Garbage collector end: collected %d objects." % (collected))

            gc.get_count()
            gc.collect()
            gc.get_count()

        print("Modeling time: ", str(round(time.time()-I_Modelado,2))+'\n')
    else:

        ModStr=str(
          "#############################################################################\n"+
          "Candidatos file has been imported (Not modelling)"+
          "#############################################################################\n"+
          "Start time again: "+ str(datetime.now()) +"\n"+
          "Folder Name: "+ str(Conid_exp) +"\n"+
          "#############################################################################\n"+
          "#############################################################################\n"
          )

        with open(ResumenEjecucion) as f:
            if not 'Candidatos file has been imported' in f.read():
                with open(ResumenEjecucion, 'a') as f:
                    print(ModStr, file=f)
        print(ModStr)
        if os.path.isfile(fileCandidato):
            Candidatos=pd.read_csv(fileCandidato, index_col=0)
###########################################################################################################
###########################################################################################################
#### Forecasting ####

    I_Forecasting=time.time()
    filenamedistKMHyb=path+path_results+"DistKMhyb.xlsx"
    # RMSE = pd.DataFrame(columns=["FileName","Rmse_Epsilon","Hill", "fore_set"])


    if not os.path.isfile(filenamedistKMHyb):
        DistKMhybF = pd.DataFrame(columns=["Valida"])
    else:
        DistKMhybF = pd.read_excel(filenamedistKMHyb,engine='openpyxl', index_col=0)

#Create file "LeerPesosdetxt.h"

    if not os.path.isfile(path_path_models+"LeerPesosdetxt.h"):
        FAVCI.LeerPesosdetxt("",path_path_models)

    for m in Candidatos.index:
        mini=time.time()
        lista=Candidatos.iloc[m,Candidatos.columns == 'Candidato'].values[0]
        FileName = "modelo"+str(m)
        Mod=tf.keras.models.load_model(path_path_models+FileName+".h5", compile=False)

        #Extract Model information
        NCapas,Act,K=FAVCI.ExtractModInfo(Mod,ResumenEjecucion)

        for fores in fore_set:
            filenamedistKMAppr=path+path_results+"DistKMAppr"+fores+".xlsx"
            filenamedistKMApprBest=path+path_results+"DistKMApprBest"+fores+".xlsx"
            Valida=str(lista)+"_"+fores+"_m_"+str(m)
            if not Valida in list(DistKMhybF["Valida"]):
                ini=time.time()
                fore_seq,fore_test,obs_test,appr_test,fore_trend_seq,Reales,Reales_trend=FAVCI.Select_fore_set(fores,
                ind_train,nfolds,ind_valid0,train_seq_num,input_num,output_num,
                ind_test0,valid_seq_num,error,errorBU,error_trend,ind_valid,ind_test,obs,appr,var_model)

                #Export input files for C++ execution
                FAVCI.ExportDataInputCMasMas("",path_path_models,fore_seq,input_num,K,Mod)
    ###########################################################################################################
                Epsilon=Reales.loc[input_num:,var_model]
                Epsilon.to_csv(path+path_results+"Epsilon"+fores+".csv")

                #creando modelo c++
                #Crea archivo LeerPesosdetxt.h
                Inicio=time.time()
                if not os.path.isfile(path_path_models+"LeerPesosdetxt.h"):
                    FAVCI.LeerPesosdetxt("",path_path_models)


                ######################################
                #### Crea archivo RedNeuronal.cpp
                if not os.path.isfile(path_path_models+FileName+".cpp"):
                    FAVCI.creaRedNeuronalcMasMas(path_path_models+FileName+".cpp",K,Act,NCapas,Functions)
                ######################################

                ######################################
                #### Compile RedNeuronal.cpp file
                Inicio=time.time()
                Plataforma=platform.platform()
                if Plataforma.find("indows") == 1:
                    FileVerification=FileName+".exe"
                else:
                    FileVerification=FileName
                if not os.path.isfile(path+path_cpp+FileVerification):
                    cwd=os.getcwd()
                    cwd2=cwd
                    os.chdir(os.path.abspath(path_path_models))

                    #O1, O2 or O3 can be used. As it increases it loses accuracy.
                    if Plataforma.find("indows") == 1:
                        pl=subprocess.Popen(r"g++ -O2 -I EINGEN/eigen/ "+FileName+ ".cpp -o "+FileName,shell=True)
                    else:
                        pl=subprocess.Popen(r"g++ -std=c++0x -O2 -I EINGEN/eigen/ "+FileName+ ".cpp -o "+FileName,shell=True)
                    pl.wait()
                    os.chdir(cwd2)
                ##############################################

                fore_timeI=time.time()
                DistKMAppr,DistKMApprBest,DistKMhyb,Fore,Cols=FAVCI.forec(t_dist,fore_seq,obs_test,appr_test, Mod,fore_test,m,input_num,
                          output_num,var_model,var_angular,var_set,FileName,Functions,"",path_path_models)
                fin=time.time()

                #RMSE:

                Epsilon_Fore=Fore.loc[input_num:,var_model]
                Epsilon_Fore.to_csv(path+path_results+"Epsilon_Fore"+fores+str(m)+".csv")
                Rmse_Epsilon=FAVCI.RMSE(Epsilon, Epsilon_Fore)


                print("Modeling + forecasting time: ", str(round(fin-ini,2))+'\n')
    ###########################################################################################################
                DistKMhyb[DistKMhyb == 0] = np.nan
                DistKMhyb["Avg_Days_Km"]=DistKMhyb[Cols].mean(axis=1,skipna = True)
                DistKMhyb["fore_set"]=fores
                DistKMhyb["Valida"]=Valida
                DistKMhyb["Valida"]=Valida
                DistKMhyb["Time"]=round(time.time()-fore_timeI,2)
                DistKMhyb["Rmse_Epsilon"]=Rmse_Epsilon


                if not os.path.isfile(filenamedistKMAppr):
                    DistKMAppr=DistKMAppr.to_excel(filenamedistKMAppr, index = True)
                if not os.path.isfile(filenamedistKMApprBest):
                    DistKMApprBest=DistKMApprBest.to_excel(filenamedistKMApprBest, index = True)

                if (rem_trend):
                    Fore[var_model]  = Fore[var_model] + fore_trend_seq[var_model]
                    Reales           = Reales_trend

                DistKMhybF=DistKMhybF.append(DistKMhyb)
                DistKMhybF.to_excel(filenamedistKMHyb, index = True)

                nombregrafico=path+path_plot+id_expM +"_"+fores+"_m_"+str(m)
                if not os.path.isfile(nombregrafico):
                    FAVCI.plot_fore(Fore,Reales,nombregrafico,plot_width,plot_height,input_num,output_num,var_model,var_lab_unit)

                if os.path.isfile(path_path_models+"Entrada.csv"):
                    os.remove(path_path_models+"Entrada.csv")
                if os.path.isfile(path_path_models+'NumeroDatos.txt'):
                    os.remove(path_path_models+'NumeroDatos.txt')
                if os.path.isfile(path_path_models+"Fore.dat"):
                    os.remove(path_path_models+"Fore.dat")

                collected = gc.collect()
                print("Garbage collector FORECASTING: collected %d objects." % (collected))

                gc.get_count()
                gc.collect()
                gc.get_count()

        #Del c++ inputs files
        for k in K:
            a_file = path_path_models+k+".csv"
            if os.path.isfile(a_file):
                os.remove(a_file)
        gc.get_count()
        gc.collect()
        gc.get_count()

    print("Fore time: ", str(round(time.time()-I_Forecasting,2))+'\n')

    #Eliminar carpeta Eigen
    if os.path.exists(path_path_models+"EINGEN"):
        try:
            shutil.rmtree(path_path_models+"EINGEN")
            print('EINGEN directory deleted')
        except OSError as err:
            print("Except: \n")
            print(err)



################################################################################################
################################################################################################
### SUMMARY:
################################################################################################
################################################################################################

    if not method==3 and Forecast==True:
        with open(ResumenEjecucion) as f:
            if 'BEST MODEL' in f.read():
                # Delete some text in a text file
                orig_file = open(ResumenEjecucion, "r")
                lines = orig_file.readlines()

                start = False
                saved_list = []
                for rec in lines:
                    if "BEST MODEL:" in rec:
                        start = True
                    if not start:
                        saved_list.append(rec)
                    if "BEST MODEL END" in rec:
                        start= False

                new_text_file = open(ResumenEjecucion, "w")
                new_text_file.writelines(saved_list)
                new_text_file.close()

        Mejor=pd.DataFrame(Candidatos.sort_values(by=['errorAvgDays'])).head(1)
        try:
            Resumen=str(
              "#############################################################################\n"+
              "BEST MODEL: \n"+
              "#############################################################################\n"+
              "The total time spent performing the gird search "+ Metodo +"of "+str(Candidatos.shape[0])+" models is: "+ str(sum([Candidatos["time"][a] for a in Candidatos.index if type(Candidatos["time"][a])!=str]))+ " seconds \n"+
              "The total forecast time is: "+ str(sum(DistKMhybF["Time"]))+ " seconds \n"+
              "#############################################################################\n"+
              "The best candidate is: \n" + str(Mejor["Candidato"].values[0]) +"\n"+
              "Such as: \n"+
              "Error" + Mejor["Candidato"].values[0]["loss"] + " is: "+ str(Mejor["Error"].values[0])+"\n"+
              "Average error in distance is: "+str(Mejor["errorAvgDays"].values[0])+"\n"
              "Number of epochs is : "+str(Mejor["Epochs"].values[0])+"\n"+
              "Model name is "+str(Mejor["Nombre"].values[0])+"\n"+
              "End time: "+ str(datetime.now()) +"\n"+
              "#############################################################################\n"+
              "#############################################################################\n"+
              "#############################################################################\n"+
              "BEST MODEL END \n")
        except:
            Resumen=str(
              "#############################################################################\n"+
              "BEST MODEL: \n"+
              "#############################################################################\n"+
              "The total time spent performing the grid search  "+ Metodo +"of "+str(Candidatos.shape[0])+" models is: "+ str(sum([Candidatos["time"][a] for a in Candidatos.index if type(Candidatos["time"][a])!=str]))+ " seconds \n"+
              "The total forecast time is: "+ str(sum(DistKMhybF["Time"]))+ " seconds \n"+
              "#############################################################################\n"+
              "The best candidate is: \n" + str(Mejor["Candidato"].values[0]) +"\n"+
              "Such as: \n"+
              "Error" + re.search('%s(.*)%s' % ("'loss':", ", 'LR'"), Mejor["Candidato"].values[0]).group(1) + " is: "+ str(Mejor["Error"].values[0])+"\n"+
              "Average error in distance is: "+str(Mejor["errorAvgDays"].values[0])+"\n"
              "Number of epochs is : "+str(Mejor["Epochs"].values[0])+"\n"+
              "Model name is "+str(Mejor["Nombre"].values[0])+"\n"+
              "End time: "+ str(datetime.now()) +"\n"+
              "#############################################################################\n"+
              "#############################################################################\n"+
              "#############################################################################\n"+
              "BEST MODEL END \n")
        with open(ResumenEjecucion, 'a') as f:
            print(Resumen
          , file=f)
        print(Resumen)

print("Hora inicio:")
print("Hora fin:")
print(datetime.now())


#Eliminar carpeta Eigen
if os.path.exists(path+path_cpp+"EINGEN"):
    try:
        shutil.rmtree(path+path_cpp+"EINGEN")
        print('EINGEN directory deleted')
    except OSError as err:
        print("Except: \n")
        print(err)




In [ ]:
os.chdir("/content/H_HSGP4_Ariadna/KerasPruebasFinal/hill1Prueba_Fachl0.5MeCartesianTrS0VaS0TeS0Res10Nf0RtFalseVarShillEp500St60VarMtheta/cpp")

In [ ]:
import subprocess


In [ ]:
run='g++ aa.cpp -o traincpp0'
pl=subprocess.Popen(run,shell=True)
print(f"Ejecución: {pl.wait()}")

In [ ]:
!g++ -std=c++0x -O2 -I "/content/H_HSGP4_Ariadna/KerasPruebasFinal/hill1Prueba_Fachl0.5MeCartesianTrS0VaS0TeS0Res10Nf0RtFalseVarShillEp500St60VarMtheta/cpp/eigen" aa.cpp -o traincpp0